# --- Cuaderno para el entrenamiento y fine tuning del clasificador de razas de perros ---
# --- Este cuaderno tambien genera un modelo optimizado usando Tensorflow Lite ---

# --- Bloque 1: Instalación y Carga de Librerías ---
# Asegúrate de que el entorno de ejecución (Runtime) en Colab esté configurado para GPU.
# Ve a 'Entorno de ejecución' > 'Cambiar tipo de entorno de ejecución' > 'Acelerador de hardware: GPU'.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import os

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Disponible: {tf.config.list_physical_devices('GPU')}")

TensorFlow Version: 2.18.0
GPU Disponible: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]



# --- Bloque 2: Parámetros y Carga/Preprocesamiento del Dataset ---
# Define los parámetros clave y carga el dataset `stanford_dogs`.

In [ ]:
IMG_SIZE = 224 # Tamaño de entrada para MobileNetV2. Las imágenes se redimensionarán a este tamaño.
BATCH_SIZE = 32 # Número de imágenes procesadas por lote durante el entrenamiento.
SHUFFLE_BUFFER_SIZE = 1000 # Tamaño del buffer para mezclar el dataset, ayuda a la aleatoriedad de los lotes.

# Cargar el dataset stanford_dogs
# `split=['train', 'test']` divide el dataset en conjuntos de entrenamiento y prueba.
# `shuffle_files=True` asegura que los archivos se mezclen antes de cargar.
# `with_info=True` devuelve metadatos del dataset, como el número y nombres de las clases.
# `as_supervised=True` devuelve el dataset en formato (imagen, etiqueta) directamente.
(ds_train, ds_test), ds_info = tfds.load(
    'stanford_dogs',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
    as_supervised=True,
)

# Obtener el número total de clases (razas de perros) del dataset.
num_classes = ds_info.features['label'].num_classes
# Obtener los nombres de las razas. Es crucial guardar esta lista para la aplicación móvil.
class_names = ds_info.features['label'].names
print(f"Número de clases (razas de perros): {num_classes}")
print(f"Nombres de las clases (primeras 10): {class_names[:10]}...") # Imprime solo las primeras 10 para no saturar

# --- Función de Preprocesamiento de Imágenes ---
def preprocess_image(image, label):
    """
    Redimensiona la imagen y la normaliza para que los valores de los píxeles estén en el rango [-1, 1].
    MobileNetV2 espera entradas en este rango.
    """
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) # Redimensiona la imagen al tamaño requerido por el modelo.
    image = tf.cast(image, tf.float32) / 127.5 - 1 # Convierte a float32 y normaliza.
    return image, label

# Aplicar la función de preprocesamiento a los datasets.
# `num_parallel_calls=tf.data.AUTOTUNE` permite que TensorFlow procese las imágenes en paralelo.
ds_train = ds_train.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
# Mezclar el dataset de entrenamiento, agrupar en lotes y precargar para optimizar el rendimiento.
ds_train = ds_train.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
# Agrupar el dataset de prueba en lotes y precargar.
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


Número de clases (razas de perros): 120
Nombres de las clases (primeras 10): ['n02085620-chihuahua', 'n02085782-japanese_spaniel', 'n02085936-maltese_dog', 'n02086079-pekinese', 'n02086240-shih-tzu', 'n02086646-blenheim_spaniel', 'n02086910-papillon', 'n02087046-toy_terrier', 'n02087394-rhodesian_ridgeback', 'n02088094-afghan_hound']...


# --- Bloque 3: Construcción del Modelo (Fine-tuning con MobileNetV2) ---
# Aquí se define la arquitectura del modelo, utilizando MobileNetV2 como base.

In [ ]:
# Cargar el modelo base pre-entrenado de MobileNetV2 desde Keras Applications.
# `input_shape`: Define el tamaño de las imágenes de entrada (224x224 con 3 canales de color).
# `include_top=False`: Excluye la capa clasificadora final del modelo original (ImageNet).
# `weights='imagenet'`: Usa los pesos pre-entrenados en el gran dataset ImageNet.
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')

# Congelar las capas convolucionales base del modelo.
# Esto significa que sus pesos no se actualizarán durante la primera fase del entrenamiento.
# Solo entrenaremos las nuevas capas que añadiremos.
base_model.trainable = False

# --- Construir el nuevo modelo añadiendo capas personalizadas ---
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3)) # Capa de entrada del modelo.
x = base_model(inputs, training=False) # Pasa las entradas a través del modelo base (importante: `training=False` para Batch Normalization).
x = tf.keras.layers.GlobalAveragePooling2D()(x) # Reduce el tensor a un vector promediando los valores espacialmente.
x = tf.keras.layers.Dropout(0.2)(x) # Capa de Dropout para regularización, ayuda a prevenir el sobreajuste.
outputs = tf.keras.layers.Dense(num_classes)(x) # Capa de salida con el número de clases de razas de perros.
                                               # No se aplica Softmax aquí, se maneja en la función de pérdida.

# Crear el modelo final combinando las entradas y salidas.
model = tf.keras.Model(inputs, outputs)

# Compilar el modelo.
# `optimizer`: Algoritmo de optimización (Adam es una buena opción general).
# `loss`: Función de pérdida. SparseCategoricalCrossentropy es adecuada para clasificación multiclase con etiquetas enteras.
#         `from_logits=True` indica que la capa de salida no tiene una activación Softmax.
# `metrics`: Métrica para monitorear durante el entrenamiento (precisión).
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print("\n--- Resumen del modelo antes del entrenamiento ---")
model.summary()


--- Resumen del modelo antes del entrenamiento ---


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 120)            │       153,720 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,411,704 (9.20 MB)

 Trainable params: 153,720 (600.47 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

# --- Bloque 4: Entrenamiento del Modelo (Fase 1: Capas Nuevas Congeladas) ---
# En esta fase, solo se entrenan las capas que añadimos (Pooling, Dropout, Dense).

In [ ]:
initial_epochs = 10 # Número de épocas para la primera fase de entrenamiento. Puedes ajustar este valor.

print(f"\n--- Iniciando entrenamiento (Fase 1) por {initial_epochs} épocas ---")
history = model.fit(ds_train,
                    epochs=initial_epochs,
                    validation_data=ds_test)



--- Iniciando entrenamiento (Fase 1) por 10 épocas ---
Epoch 1/10


# --- Bloque 5: Descongelar y Entrenar (Fase 2: Fine-tuning) ---
# Aquí se permite que algunas capas del modelo base también se entrenen, afinando el modelo.


In [ ]:
base_model.trainable = True # Hacer todas las capas del modelo base entrenables.

# Iterar sobre las capas del modelo base y congelar las que no queremos afinar.
# Un valor común es descongelar la última parte del modelo base, por ejemplo, las últimas 50-100 capas.
# Ajusta `fine_tune_at` para controlar cuántas capas se entrenan del modelo base.
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompilar el modelo después de cambiar la entrenabilidad de las capas.
# Es fundamental recompilar para que los cambios en `trainable` surtan efecto.
# Usamos un learning rate mucho más bajo para el fine-tuning para evitar corromper los pesos pre-entrenados.
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print("\n--- Resumen del modelo después de descongelar capas para fine-tuning ---")
model.summary()

# --- Entrenamiento del modelo (Fase 2: Fine-tuning) ---
fine_tune_epochs = 10 # Número de épocas para la fase de fine-tuning.
total_epochs = initial_epochs + fine_tune_epochs # Épocas totales combinadas.

print(f"\n--- Iniciando fine-tuning (Fase 2) por {fine_tune_epochs} épocas adicionales ---")
history_fine_tune = model.fit(ds_train,
                              epochs=total_epochs,
                              initial_epoch=history.epoch[-1], # Continúa el entrenamiento desde la última época de la Fase 1.
                              validation_data=ds_test)

# --- Bloque 6: Evaluación del Modelo y Visualización ---
# Combina los datos de historial de ambas fases para una visualización completa del progreso.

In [ ]:
acc = history.history['accuracy'] + history_fine_tune.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine_tune.history['val_accuracy']

loss = history.history['loss'] + history_fine_tune.history['loss']
val_loss = history.history['val_loss'] + history_fine_tune.history['val_loss']

# Graficar la precisión de entrenamiento y validación
plt.figure(figsize=(10, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Precisión de Entrenamiento')
plt.plot(val_acc, label='Precisión de Validación')
plt.ylim([0.0, 1])
# Línea vertical para marcar el inicio del fine-tuning
plt.plot([initial_epochs-1, initial_epochs-1],
          plt.ylim(), label='Inicio Fine-tuning')
plt.legend(loc='lower right')
plt.title('Precisión de Entrenamiento y Validación')

# Graficar la pérdida de entrenamiento y validación
plt.subplot(2, 1, 2)
plt.plot(loss, label='Pérdida de Entrenamiento')
plt.plot(val_loss, label='Pérdida de Validación')
plt.ylim([0, 1.0])
# Línea vertical para marcar el inicio del fine-tuning
plt.plot([initial_epochs-1, initial_epochs-1],
         plt.ylim(), label='Inicio Fine-tuning')
plt.legend(loc='upper right')
plt.title('Pérdida de Entrenamiento y Validación')
plt.xlabel('Época')
plt.tight_layout()
plt.show()

# Evaluación final del modelo en el set de prueba
loss, accuracy = model.evaluate(ds_test)
print(f"\nPrecisión final en el set de prueba: {accuracy:.4f}")

# --- Bloque 7: Conversión a TensorFlow Lite y Descarga ---
# Este bloque convierte el modelo entrenado a un archivo `.tflite` y guarda las etiquetas.


In [ ]:
# Para TFLite, generalmente queremos que el modelo de salida dé probabilidades (softmax).
# Durante el entrenamiento, usamos `from_logits=True` en la función de pérdida para mayor estabilidad numérica.
# Aquí, creamos un modelo temporal que incluye una capa Softmax al final.
model_for_tflite = tf.keras.Sequential([
    model, # Tu modelo entrenado
    tf.keras.layers.Softmax() # Añade la capa Softmax para obtener probabilidades
])

# Crear el convertidor de TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_tflite)

# Optimización (opcional pero muy recomendable para dispositivos móviles)
# `tf.lite.Optimize.DEFAULT` aplica optimizaciones como la cuantificación de pesos (a 8 bits por defecto),
# lo que reduce el tamaño del modelo y acelera la inferencia.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert() # Realiza la conversión.

# Guardar el modelo .tflite en un directorio.
tflite_models_dir = 'tflite_models'
os.makedirs(tflite_models_dir, exist_ok=True) # Crea el directorio si no existe.
tflite_model_path = os.path.join(tflite_models_dir, 'stanford_dogs_classifier.tflite')

with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"\nModelo TFLite guardado en: {tflite_model_path}")

# --- Guardar los Nombres de las Clases ---
# Es CRUCIAL que este archivo de etiquetas se incluya en tu aplicación móvil
# y se lea en el mismo orden para mapear las predicciones numéricas a los nombres de las razas.
labels_path = os.path.join(tflite_models_dir, 'labels.txt')
with open(labels_path, 'w') as f:
    for name in class_names:
        f.write(f"{name}\n")

print(f"Archivo de etiquetas guardado en: {labels_path}")

# --- Descargar los Archivos ---
# Esta celda te permitirá descargar el modelo .tflite y el archivo de etiquetas directamente desde Colab.
from google.colab import files
print("\nPreparando la descarga de los archivos...")
files.download(tflite_model_path)
files.download(labels_path)
print("¡Descarga completada! Revisa las descargas de tu navegador.")